# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [9]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [3]:
!pip3 install cassandra-driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 KB 3.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 2.2 MB/s eta 0:00:00a 0:00:01
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [2687 lines of output]
      /Users/rojinashrestha/miniforge3/envs/mlp/lib/python3.8/site-packages/setuptools/installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
        warnings.warn(
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-11.0-arm64-cpython-38
      creating build/lib.macosx-11.0-arm64-cpython-38/cassandra
      copying cassandra/marshal.py -> build/lib.macosx-11.0-arm64-cpython-38/cassandra
      copying cassandra/auth.py -> build/lib.macosx-11.0-arm64-cpython-38/cassandra
     

Failed to build cassandra-driver
  Running setup.py install for cassandra-driver ... done
  DEPRECATION: cassandra-driver was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. Discussion can be found at https://github.com/pypa/pip/issues/8368


#### Creating list of filepaths to process original event csv data files

In [5]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/rojinashrestha/Data Modeling with Cassandra/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [6]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [58]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [27]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['192.168.0.1', '192.168.0.2'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

NoHostAvailable: ('Unable to connect to any servers', {'192.168.0.1:9042': ConnectionRefusedError(61, "Tried connecting to [('192.168.0.1', 9042)]. Last error: Connection refused"), '192.168.0.2:9042': OSError(64, "Tried connecting to [('192.168.0.2', 9042)]. Last error: Host is down")})

#### Create Keyspace

In [60]:
# TO-DO: Create a Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = {'class' : 'SimpleStrategy','replication_factor':1}

""")

#### Set Keyspace

In [61]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [62]:
# The table **sessions**, the **sessionId** column was used as a PARTITION KEY to filter the result by this column. We have used **itemInSession** column as clustering columns to sort the result.

query = "CREATE TABLE IF NOT EXISTS sessions"
query = query +"(sessionId int,itemInSession int,length float,artist text,song text, PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [63]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO sessions (sessionId, itemInSession, length,artist,song)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"

        session.execute(query, (int(line[8]),int(line[3]),float(line[5]),line[0],line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [85]:

query= "SELECT artist,song,length FROM sessions WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
    print(rows)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist,row.song,row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [87]:
# The table **artist_song_user**, the **userid** and **sessionId** column was used as a PARTITION KEY to filter the result by this column. We have used **itemInSession** column as clustering columns to sort the result.

query = "CREATE TABLE IF NOT EXISTS artist_song_user"
query = query +"(userid int,sessionid int,artist text,song text,itemInSession int,firstName text,lastName text, PRIMARY KEY((userid,sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)




                    

In [88]:

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO artist_song_user (userid,sessionid,artist,song,itemInSession,firstName,lastName)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"

        session.execute(query, (int(line[10]),int(line[8]),line[0],line[9],int(line[3]),line[1],line[4]))

In [93]:
        
query = "SELECT artist,song,firstName,lastName FROM artist_song_user WHERE userid=10 and sessionId=182"
try:
    artists = session.execute(query)

except Exception as e:
    print(e)

for artist in artists:
 
    print(artist.artist,artist.song,artist.firstname,artist.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [95]:
# The table **user_song**, the **song**  column was used as a PARTITION KEY to filter the result by this column. We have used **user_id** column as clustering columns to sort the result.
query = "CREATE TABLE IF NOT EXISTS user_song"
query = query +"(song text,user_id int,firstname text,lastname text, PRIMARY KEY((song)), user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
             

In [97]:

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO user_song (song ,user_id ,firstname ,lastname )"
        query = query + "VALUES (%s,%s,%s,%s)"

        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

In [101]:
        
query = "SELECT firstname,lastname FROM user_song WHERE  song='All Hands Against His Own'"
try:
    songs = session.execute(query)
  
except Exception as e:
    print(e)

for song in songs:
 
    print(song.firstname,song.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [ ]:
query = "DROP TABLE sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query2 = "DROP TABLE artist_song_user"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
query3 = "DROP TABLE user_song"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()